# AWS KMS - API Walkthrough

### Objective
The Objective of this notebook is to take you on an interactive journey through AWS KMS and its API. This notebook aims to delve into: 
* AWS KMS Usage in real-world applications
* KMS API Deep-dive, including new features released by AWS
* Explain complex concepts with simple, interactive workflows

### Requirements
You will need: 
* An (activated) AWS Account
* Python 3.6.X
* Stuff in the `requirements.txt` file
* Jupyter Notebook

### Let's start with learning about Key Generation with AWS - KMS

Let's generate a Customer-Master-Key for a particular use-case. Let's generate, in this case, an AES, Symmetric Customer Master Key (CMK) for an application we're building. 

#### Remember: 
* You can generate Symmetric and Asymmetric Master Keys in KMS
* Keys never leave the KMS unencrypted
* In the case of Asymmetric Keys, the Private Key never leaves AWS. 

**Asymmetric CMK generation is not allowed in all regions**

In [ ]:
# utils - Just some utility functions that we'll be using for the rest of this tutorial
import boto3
from botocore.exceptions import ClientError
from loguru import logger
from sys import stdout, stderr
logger.add(stderr, colorize = True, format = "<red>{time}</red> <level>{message}</level>", level="ERROR")
logger.add(stdout, colorize = True, format = "<green>{time}</green> <level>{message}</level>", level="INFO")
keyclient = boto3.client("kms", region_name="us-east-1")

logger.info("Initialized utils")

### Generate CMK
[Boto3 Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/kms.html#KMS.Client.create_key)

In [ ]:
def generate_symmetric_cmk():
    """
    This function generates a symmetric Customer Master key to be used in AWS
    """
    try:
        response = keyclient.create_key(
            Description = 'This is a Symmetric CMK we are generating for use with our app',
            KeyUsage = "ENCRYPT_DECRYPT", #if using Asymmetric you need to use "SIGN_VERIFY"
            CustomerMasterKeySpec = "SYMMETRIC_DEFAULT", #this is AES-256-GCM
            Origin = "AWS_KMS", #this can be "EXTERNAL" when you import stuff. or "AWS_CLOUDHSM" if you use that as the backend
            Tags = [
                {
                    "TagKey": "appname",
                    "TagValue": "mynewapp-encrypt-key"
                }
            ]  
        )
        if response['KeyMetadata']['Arn']:
            symmetric_cmk = {
                "Arn": response['KeyMetadata']['Arn'],
                "KeyId": response['KeyMetadata']['KeyId']
            }
            return symmetric_cmk
        
    except ClientError as e:
        logger.error("Unable to generate key")
        logger.error(e)

symmetric_cmk = generate_symmetric_cmk()


Now lets use the CMK for a bunch of things we need, in terms of encryption and decryption

### Envelope Encryption
Envelope Key is a Data Key that is generated by the CMK to be used to encrypt datasets. This ensures that the Data Key is used to encrypt/decrypt the data whereas the CMK is used as a wrapper key, to wrap the Data Key. 
This can be generated with or without the plaintext version of the Data. 

**You should only be storing the Encrypted Data Key, in your environment. Never store the Plaintext Data Key**

[Boto3 Docs - Generate Data Key](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/kms.html#KMS.Client.generate_data_key)

In [ ]:
# Envelope Encryption
datakey_response = keyclient.generate_data_key(
    KeyId = symmetric_cmk['KeyId'], #this can be the CMK's ARN also OR Alias Name and ARN
    EncryptionContext = {
        "name":"mynewapp-data-encrypt-key"
    },
    KeySpec = "AES_256"
    # you can either use `KeySpec` param or the `NumberOfBytes` param to define Keylength. 
    # NumberOfBytes is between 64 and 512
)
from base64 import b64encode
datakey_response['Plaintext'] = b64encode(datakey_response['Plaintext']).decode()
datakey_response['CiphertextBlob'] = b64encode(datakey_response['CiphertextBlob']).decode()
logger.info("Data Key: {}".format(datakey_response))


### Let's encrypt and decrypt data with the Envelope Key

Now that we have a key that we can use, which is low-latency and useful for faster encrypts and decrypts; Let's use it. 

#### Encrypting Data with Python's Pycryptodome library

In [ ]:
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64decode

plaintext = b"Hello, this is some plaintext data"
# we're going to encrypt some data with AES CFB
ptkey = b64decode(datakey_response["Plaintext"].encode())
iv = get_random_bytes(16)

new_cipher = AES.new(ptkey, AES.MODE_CFB, iv)
ciphertext = b64encode(new_cipher.encrypt(pad(plaintext,16))).decode()

logger.info("Ciphertext generated from Data Key: {}".format(ciphertext))

#### Let's try and decrypt this data

In [ ]:
dcipher = AES.new(ptkey, AES.MODE_CFB, iv)
plaintext_value = unpad(dcipher.decrypt(b64decode(ciphertext.encode())), 16).decode()
logger.info("Plaintext after decryption: '{}'".format(plaintext_value))

Aside from using DataKeys, its also possible to directly use CMKs to encrypt/decrypt data. 

Should you use it that way? It depends. There are some pros and cons.
#### Pros
* No need to worry about Key rotation when using the CMK. CMK can be setup pretty easily for key rotation
* No need to worry about additional key management, padding, etc
* Easy to manage Access to Data Key (in this case CMK)


#### Cons
* Implement per-user encryption is not possible when using the CMK directly for encryption/decryption
* Reduce IAM privileges on the CMK to decrypting only the data key. Better Privilege Control
* Encrypting data closest to the "client" is likely going to be better in terms of performance. 

### Rotating the CMK

CMK can be automatically rotated (set by schedule) or enabled after generation

In [ ]:
try:
    rotate_resp = keyclient.enable_key_rotation(
        KeyId = symmetric_cmk['KeyId']
    )
    logger.info("Succesfully enabled rotation on KeyId: {}".format(symmetric_cmk['KeyId']))
    logger.info(rotate_resp)
except ClientError as rotate_e:
    logger.error(rotate_e)

### Asymmetric Ops - CMK
Recently, AWS enabled Asymmetric Keys as CMKs within KMS. This is a welcome change, especially for use-cases involving: 
* Secure Signature => Signing messages with the Private Key (and protection of said key)
* Intermediate Asymmetric Keys => Generating Intermediate Asymmetric Data Keys (like Symmetric Data Keys)

In [ ]:
def create_asymm_key():
    """Function that generates a CMK Keypair"""
    try:
        response = keyclient.create_key(
            Description = 'This is an Asymmetric Key we are generating for our APP',
            KeyUsage = "SIGN_VERIFY",
            CustomerMasterKeySpec = "ECC_NIST_P256", #secp256r1 
            Origin = "AWS_KMS", #this can be "EXTERNAL" when you import stuff. or "AWS_CLOUDHSM" if you use that as the backend
            Tags = [
                {
                    "TagKey": "name",
                    "TagValue": "mynewapp-encrypt-asymm-cmk"
                }
            ]  
        )
        if response['KeyMetadata']['Arn']:
            asymmetric_cmk = {
                "Arn": response['KeyMetadata']['Arn'],
                "KeyId": response['KeyMetadata']['KeyId']
            }
            return asymmetric_cmk
        
    except ClientError as e:
        logger.error("Unable to generate key")
        logger.error(e)

asymm_key = create_asymm_key()
logger.info(asymm_key)

### Let's run the following Asymmetric Key Operations from now: 
* Retrieve Public Key
* Sign `payload` with Private Key and Verify with Public Key
* Generate Asymmetric Data Keys

In [ ]:
# Retreive public key

pubkey = keyclient.get_public_key(KeyId = asymm_key['KeyId'])
logger.info(pubkey)
logger.info("Public Key Value (base64 enc): {}".format(b64encode(pubkey['PublicKey']).decode()))

In [ ]:
# let's generate some json and sign it
import json
def sign_payload(some_dict):
    if isinstance(some_dict, dict):
        json_val = json.dumps(some_dict)
        try:
            json_sig = keyclient.sign(
                KeyId = asymm_key['KeyId'],
                Message = json_val.encode(),
                MessageType = "RAW",
                SigningAlgorithm=pubkey['SigningAlgorithms'][0]
            )
            signature_value = b64encode(json_sig['Signature']).decode()
            return signature_value
        except ClientError as sig_error:
            logger.error("Unable to generate signature for JSON")
            logger.error(sig.error)

signature = sign_payload({"hello": "world"})
logger.info(signature)

In [ ]:
# lets verify the signed payload against the asymmetric key

try:
    verified_sig = keyclient.verify(
        KeyId = asymm_key['KeyId'],
        Message = json.dumps({"hello": "world"}).encode(),
        MessageType = "RAW",
        Signature = signature.encode(),
        SigningAlgorithm=pubkey['SigningAlgorithms'][0]
    )
    logger.info("Signature verified.")
except ClientError:
    logger.error("Signature mismatch")